In [55]:
import sqlite3 as sql
import pandas as pd

In [56]:
# create and make a connection to the database file
conn = sql.connect('dojo.db')

In [57]:
# we will use pandas for easier file loading to the database
csv_files = ['location', 'region', 'product', 'location_region', 'sales']
for csv in csv_files:
    df = pd.read_csv(f'.//db_files//{csv}.csv')
    df.to_sql(csv, conn, if_exists='replace', index=False)

In [58]:
# cursor is what is used to "drive" queries
c = conn.cursor()

In [59]:
# now that we have the database set up, we can query our data

# simple query to pull all locations and their regions
c.execute('select * from location_region')
# fetchall retreives the results of the most recent execute statement
print(c.fetchall())

[('L001', 'R010'), ('L002', 'R005'), ('L003', 'R009'), ('L004', 'R001'), ('L005', 'R005'), ('L006', 'R006'), ('L007', 'R003'), ('L008', 'R003'), ('L009', 'R010'), ('L010', 'R010'), ('L011', 'R001'), ('L012', 'R005'), ('L013', 'R002'), ('L014', 'R002'), ('L015', 'R004'), ('L016', 'R004'), ('L017', 'R003'), ('L018', 'R009'), ('L019', 'R006'), ('L020', 'R003'), ('L021', 'R006'), ('L022', 'R002'), ('L023', 'R002'), ('L024', 'R010'), ('L025', 'R004'), ('L026', 'R005'), ('L027', 'R004'), ('L028', 'R001'), ('L029', 'R006'), ('L030', 'R007'), ('L031', 'R005'), ('L032', 'R007'), ('L033', 'R003'), ('L034', 'R004'), ('L035', 'R002'), ('L036', 'R009'), ('L037', 'R008'), ('L038', 'R007'), ('L039', 'R010'), ('L040', 'R006'), ('L041', 'R010'), ('L042', 'R004'), ('L043', 'R003'), ('L044', 'R009'), ('L045', 'R005'), ('L046', 'R006'), ('L047', 'R003'), ('L048', 'R008'), ('L049', 'R003'), ('L050', 'R002'), ('L051', 'R005')]


In [60]:
# let's pull the names for the locations and regions instead of the ids
# for this we need to join the location, region, and location_region table 
# and use the where clause to show the feild that are the same 
c.execute("""
    select location_name, region_name
    from location join region join location_region
    where location.location_id == location_region.location_id
        and region.region_id == location_region.region_id    
""")
print(c.fetchall())

[('Alabama', 'Southeast'), ('Arizona', 'Mountain West'), ('Arkansas', 'South Central'), ('California', 'Far West'), ('Colorado', 'Mountain West'), ('Connecticut', 'New England'), ('Delaware', 'Midsouth'), ('District of Columbia', 'Midsouth'), ('Florida', 'Southeast'), ('Georgia', 'Southeast'), ('Hawaii', 'Far West'), ('Idaho', 'Mountain West'), ('Illinois', 'Great Lakes'), ('Indiana', 'Great Lakes'), ('Iowa', 'Midwest'), ('Kansas', 'Midwest'), ('Kentucky', 'Midsouth'), ('Louisiana', 'South Central'), ('Maine', 'New England'), ('Maryland', 'Midsouth'), ('Massachusetts', 'New England'), ('Michigan', 'Great Lakes'), ('Minnesota', 'Great Lakes'), ('Mississippi', 'Southeast'), ('Missouri', 'Midwest'), ('Montana', 'Mountain West'), ('Nebraska', 'Midwest'), ('Nevada', 'Far West'), ('New Hampshire', 'New England'), ('New Jersey', 'Northeast'), ('New Mexico', 'Mountain West'), ('New York', 'Northeast'), ('North Carolina', 'Midsouth'), ('North Dakota', 'Midwest'), ('Ohio', 'Great Lakes'), ('Okla

In [61]:
# now we will look at some sales data
# similar to the last query we will join the relevant tables and 
# show which field are the same
c.execute("""
    select location_name, product_name, lbs, price_per_lb
    from location join location_region join sales join product
    where location.location_id == location_region.location_id
        and location.location_id == sales.location_id
        and sales.product_id = product.product_id 
""")
print(c.fetchmany(20)) # just pull the first 20 (not the top 20; we will do that next)

[('Alabama', 'Alfalfa pellets', 707.2, 4.68), ('Alabama', 'Apples, dried', 639.3, 4.63), ('Alabama', 'Chestnuts', 828.9, 1.07), ('Alabama', 'Chilled meat', 158.0, 1.21), ('Alabama', 'Corn', 712.1, 4.38), ('Alabama', 'Fabrics/cloth/textiles', 2.2, 3.05), ('Alabama', 'Flax tow', 916.7, 3.13), ('Alabama', 'Household appliances/white goods', 875.4, 1.84), ('Alabama', 'Kiwifruit', 843.3, 2.56), ('Alabama', 'Onions', 790.0, 4.46), ('Alabama', 'Palm kernel oil', 744.9, 2.89), ('Alabama', 'Peanut expeller', 795.4, 5.43), ('Alabama', 'Potatoes', 334.1, 1.76), ('Alabama', 'Raw sugar', 204.0, 3.26), ('Alabama', 'Roofing felt', 436.3, 2.61), ('Alabama', 'Sesame seed expeller', 88.4, 3.17), ('Alabama', 'Sunflower pellets', 83.4, 4.58), ('Alabama', 'Sunflower pellets', 972.8, 4.58), ('Alabama', 'Wool', 442.3, 4.5), ('Arizona', 'Avocados', 159.6, 2.62)]


In [62]:
# let's now look at the total price and sort the values
c.execute("""
    select location_name, product_name, lbs, price_per_lb, (lbs * price_per_lb) price
    from location join location_region join sales join product
    where location.location_id == location_region.location_id
        and location.location_id == sales.location_id
        and sales.product_id = product.product_id 
    order by price desc
""")
print(c.fetchmany(20)) # just pull the first 20 (not the top 20; we will do that next)

[('Nebraska', 'Ramie', 989.9, 5.99, 5929.501), ('Georgia', 'Limes', 988.0, 5.94, 5868.72), ('Michigan', 'Copra expeller', 975.8, 5.98, 5835.284000000001), ('Mississippi', 'Ramie', 962.6, 5.99, 5765.974), ('Nebraska', 'Carpets', 969.4, 5.76, 5583.744), ('Pennsylvania', 'Sunflower seeds', 940.2, 5.71, 5368.542), ('Florida', 'Prunes', 891.7, 5.99, 5341.283), ('Montana', 'Mangos', 1000.2, 5.29, 5291.058), ('South Dakota', 'Natural rubber', 955.3, 5.42, 5177.726), ('Virginia', 'Rum', 996.0, 5.19, 5169.240000000001), ('Arkansas', 'Pipes', 831.2, 5.99, 4978.888000000001), ('California', 'Natural rubber', 918.0, 5.42, 4975.5599999999995), ('North Carolina', 'Ramie', 830.4, 5.99, 4974.0960000000005), ('Iowa', 'Coconut fiber', 844.6, 5.84, 4932.464), ('Ohio', 'Almonds', 835.7, 5.89, 4922.273), ('Oklahoma', 'Apples', 961.2, 5.11, 4911.732000000001), ('Indiana', 'Corn gluten pellets', 886.7, 5.51, 4885.717), ('Idaho', 'Coconut fiber', 832.2, 5.84, 4860.048), ('New Hampshire', 'Copra expeller', 805

In [63]:
# we can print this a little prettier
print(f'Name, Product, Price')
lines = c.fetchmany(50)
for line in lines:
    print(f'{line[0]}, {line[1]}, ${line[4]:.2f}')

Name, Product, Price
Kentucky, Candy sugar, $4776.49
Wyoming, Almonds, $4749.70
North Dakota, Almonds, $4714.36
North Carolina, Mangos, $4711.27
Indiana, Peanut expeller, $4666.54
Florida, Chocolate, solid, $4635.90
Washington, Artichokes, $4630.75
Kansas, Oranges, $4558.18
New Hampshire, Wheat bran pellets, $4504.56
Florida, Sesame oil, $4474.25
Alabama, Sunflower pellets, $4455.42
Iowa, Ramie, $4454.16
Kansas, Carpets, $4403.52
Colorado, Copra expeller, $4402.48
Wisconsin, Caraway, $4396.30
California, Mustard oil, $4364.98
Alabama, Peanut expeller, $4319.02
Arizona, Chocolate, solid, $4286.04
West Virginia, Hemp oil, $4282.20
South Carolina, Palm oil, $4253.59
Virginia, Frozen meat, $4209.03
Kentucky, Bay leaves, $4195.68
Ohio, Palm oil, $4172.94
Washington, Newsprint, $4168.63
District of Columbia, Sunflower pellets, $4136.66
Pennsylvania, Apples, dried, $4136.44
Kansas, Sunflower pellets, $4036.35
New Hampshire, Cottonseed oil, $4017.66
Puerto Rico, Peanut expeller, $3995.94
Iowa,

In [ ]:
# last query we will take some user input for a region and how the sales by product
# lets start by getting the available regions
c.execute('select region_name from region')
regions = [reg[0] for reg in c.fetchall()] # fetchall returns tuple of lists, convert to just a list
region = ''
while True:
    print('Avaiable regions: ' + ', '.join(regions)) # show the valid regions
    region = input('What input would you like to see sales data for? ') # ask user for input
    # if the input is valid, we can break the loop, else ask for input again
    if region not in regions:
        print("Invalid Input!")
    else:
        break

# query the database using the user's input
# we need to introduce the group by clause since the products are 
# associated with with locations, not regions
# we also will use the where clause to test for only the region we want
c.execute("""
    select region_name, product_name, (lbs * price_per_lb) price
    from region join location_region join sales join product
    where region.region_id == location_region.region_id
        and location_region.location_id == sales.location_id
        and sales.product_id = product.product_id 
        and region_name = ?
    group by region_name, product_name
    order by price desc
""", (region,)) # when only using one parameter in the query you need an extra comma and the end of the tuple being passed in

results = c.fetchall()
print(f'Results for {region}\nProduct, Price')
print('-'*30)
for line in results:
    print(f'{line[1]}, ${line[2]:.2f}')

In [ ]:
# last step we need is to close the connection to the database
conn.close()